In [56]:
import os
import requests
from dotenv import load_dotenv
from dataclasses import dataclass
from langchain_groq import ChatGroq
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool, ToolRuntime
from typing import Optional

load_dotenv()

True

In [57]:
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [58]:
@dataclass
class Context:
    user_id : str
    
@dataclass
class ResponseFormat:
    full_respense: str
    summary: str
    city: Optional[str]
    temprature_fahrenhite: Optional[float]
    temprature_celcius: Optional[float]
    humidity: Optional[float]


In [59]:
@tool('locate_user', description='Look up for user\'s city based on the context')
def locate_user(runtime: ToolRuntime[Context]):
    match runtime.context.user_id:
        case 'Sparsh':
            return 'Indore'
        case 'Keshav':
            return 'Bhopal'
        case 'Mahak':
            return 'Bengaluru'
        case _:
            return 'Unknown'

@tool('get_weather', description='Returns weather information for a given city', return_direct=False)
def get_weather(city: str):
    response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?unitGroup=metric&include=current&key={WEATHER_API_KEY}&contentType=json")
    return response.json()


In [60]:
model = init_chat_model(model='llama-3.3-70b-versatile', temperature=0.3, model_provider='groq')
checkpoint = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[get_weather,locate_user],
    system_prompt="""
    You are a helpful and funny weather assistant which always cracks jokes and 
    has a humourous response while also being remaining helpful.""",   
    context_schema=Context,
    response_format= ResponseFormat,
    checkpointer=checkpoint
)

In [67]:
config = {'configurable': {'thread_id': 1}}

response = agent.invoke({
    'messages': [
        {'role': 'user', 'content': 'What is the weather like?'}
    ]},
    config=config,
    context=Context(user_id='Mahak'))

In [68]:
print(response['structured_response'])

ResponseFormat(full_respense='The weather is lovely today! It is currently 24.9 degrees Celsius with a humidity of 66.7%. The conditions are partially cloudy.', summary='Partially cloudy', city='Bengaluru', temprature_fahrenhite=76.82, temprature_celcius=24.9, humidity=66.7)


In [65]:
config = {'configurable': {'thread_id': 1}}

response = agent.invoke({
    'messages': [
        {'role': 'user', 'content': 'And is this usual?'}
    ]},
    config=config,
    context=Context(user_id='Mahak'))

In [66]:
print(response['structured_response'])

ResponseFormat(full_respense='The weather is lovely today! It is currently 24.9 degrees Celsius with a humidity of 66.7%. The conditions are partially cloudy.', summary='Partially cloudy', city='Bengaluru', temprature_fahrenhite=76.82, temprature_celcius=24.9, humidity=66.7)
